<a href="https://colab.research.google.com/github/nickgreenquist/recsys/blob/main/RecSys_TFRS_Content_Based_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.4 MB/s eta 0:00:00


In [2]:
import argparse
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import pickle
import plotly.graph_objects as go
import plotly.io as pio
import re

pio.renderers.default = "browser"

In [ ]:
# Get Dataset - 100k

# movies_data_path_100k = './datasets/movies_data_100k'
# ratings_data_path_100k = './datasets/ratings_data_100k'

# movies_100k = None
# ratings_100k = None

# if os.path.exists(movies_data_path_100k) and os.path.exists(ratings_data_path_100k):
#     movies_100k = tf.data.Dataset.load(movies_data_path_100k)
#     ratings_100k = tf.data.Dataset.load(ratings_data_path_100k)
# else:
#     movies_100k = tfds.load("movielens/100k-movies", split="train").map(lambda x: {**x, 'movie_title_text': x['movie_title']})
#     ratings_100k = (tfds.load("movielens/100k-ratings", split="train")
#                                                 .map(lambda x: {**x, 'movie_title_text': x['movie_title']})
#                     .shuffle(100_000, seed=42, reshuffle_each_iteration=False))
#     tf.data.Dataset.save(movies_100k, movies_data_path_100k)
#     tf.data.Dataset.save(ratings_100k, ratings_data_path_100k)

In [9]:
# Get Dataset - 1M

movies_data_path_1M = './datasets/movies_data_1M'
ratings_data_path_1M = './datasets/ratings_data_1M'

movies_1M = None
ratings_1M = None

if os.path.exists(movies_data_path_1M) and os.path.exists(ratings_data_path_1M):
    movies_1M = tf.data.Dataset.load(movies_data_path_1M)
    ratings_1M = tf.data.Dataset.load(ratings_data_path_1M)
else:
    movies_1M = tfds.load("movielens/1m-movies", split="train").map(lambda x: {**x, 'movie_title_text': x['movie_title']})
    ratings_1M = (tfds.load("movielens/1m-ratings", split="train")
                                                .map(lambda x: {**x, 'movie_title_text': x['movie_title']})
                    .shuffle(100_000, seed=42, reshuffle_each_iteration=False))
    tf.data.Dataset.save(movies_1M, movies_data_path_1M)
    tf.data.Dataset.save(ratings_1M, ratings_data_path_1M)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3883 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1000209 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/1m-ratings/0.1.1.incomplete2EAQB0/movielens-train.tfrecord*...: …

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/1m-ratings/0.1.1. Subsequent calls will reuse this data.


In [10]:
# Choose which dataset to use

movies = movies_1M
ratings = ratings_1M

In [15]:
all_ratings = list(ratings.map(lambda x: {
    'movie_title': x["movie_title"],
    'user_id': x['user_id'],
    'user_gender': x['user_gender'],
    'bucketized_user_age': x['bucketized_user_age'],
    'user_occupation_label': x['user_occupation_label'],
    'movie_genres': x['movie_genres']
    }).apply(tf.data.experimental.dense_to_ragged_batch(len(ratings))))[0]

Instructions for updating:
Use `tf.data.Dataset.ragged_batch` instead.


In [16]:
all_movies = list(movies.apply(tf.data.experimental.dense_to_ragged_batch(len(movies))))[0]

In [17]:
unique_movie_titles = np.unique(all_movies['movie_title'])
unique_movie_genres, _ = tf.unique(all_movies['movie_genres'].flat_values)

additional_feature_info = {
    'unique_movie_genres': unique_movie_genres,
    'user_gender': all_ratings['user_gender'],
    'bucketized_user_age': all_ratings['bucketized_user_age'],
    'user_occupation_label': all_ratings['user_occupation_label']
    }

In [18]:
class UserModel(tf.keras.Model):
    def __init__(self, embedding_size=32, additional_features=(), additional_feature_info=None):
        super().__init__()
        self.additional_embeddings = {}

        if 'bucketized_user_age' in additional_features:
            self.user_age_normalizer = tf.keras.layers.Normalization(axis=None)
            self.user_age_normalizer.adapt(additional_feature_info['bucketized_user_age'])
            self.additional_embeddings['bucketized_user_age'] = tf.keras.Sequential([self.user_age_normalizer,
                                                                                     tf.keras.layers.Reshape([1])])

        if 'user_gender' in additional_features:
            self.user_gender_normalizer = tf.keras.layers.Normalization(axis=None)
            self.user_gender_normalizer.adapt(additional_feature_info['user_gender'])
            self.additional_embeddings['user_gender'] = tf.keras.Sequential([self.user_gender_normalizer,
                                                                                     tf.keras.layers.Reshape([1])])

        if 'user_occupation_label' in additional_features:
            self.user_occupation_normalizer = tf.keras.layers.Normalization(axis=None)
            self.user_occupation_normalizer.adapt(additional_feature_info['user_occupation_label'])
            self.additional_embeddings['user_occupation_label'] = tf.keras.Sequential([self.user_occupation_normalizer,
                                                                                     tf.keras.layers.Reshape([1])])

    def call(self, inputs):
        return tf.concat([self.additional_embeddings[k](inputs[k]) for k in self.additional_embeddings], axis=1)

In [19]:
class MovieModel(tf.keras.Model):
    def __init__(self, unique_movie_titles, additional_features, additional_feature_info, embedding_size=32):
        super().__init__()
        self.additional_embeddings = {}

        self.title_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_movie_titles, mask_token=None),
            tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_size)
        ])

        if 'movie_genres' in additional_features:
            self.additional_embeddings['movie_genres'] = tf.keras.Sequential([
                tf.keras.layers.Embedding(max(additional_feature_info['unique_movie_genres']) + 1, embedding_size),
                tf.keras.layers.Lambda(lambda x: tf.math.reduce_mean(x, axis=1))
            ])

        if 'movie_title_text' in additional_features:
            max_tokens = 10_000
            self.title_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_tokens)
            self.title_vectorizer.adapt(unique_movie_titles)
            self.additional_embeddings['movie_title_text'] = tf.keras.Sequential([
                self.title_vectorizer,
                tf.keras.layers.Embedding(max_tokens, embedding_size, mask_zero=True),
                tf.keras.layers.GlobalAveragePooling1D(),
            ])

    def call(self, inputs):
        return tf.concat([self.title_embedding(inputs['movie_title'])] +
                         [self.additional_embeddings[k](inputs[k]) for k in self.additional_embeddings],
                         axis=1)

In [20]:
class QueryCandidateModel(tf.keras.Model):
    def __init__(self, layer_sizes, embedding_model):
        super().__init__()
        self.embedding_model = embedding_model
        self.dense_layers = tf.keras.Sequential()
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation='relu'))
        self.dense_layers.add(tf.keras.layers.Dense(layer_sizes[-1]))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [25]:
class MovieLensModel(tfrs.models.Model):
    def __init__(self, layer_sizes, movies, unique_movie_titles, embedding_size,
                 additional_features, additional_feature_info):
        super().__init__()
        self.additional_features = additional_features
        self.query_model = QueryCandidateModel(layer_sizes, UserModel(embedding_size=embedding_size,
                                                                      additional_features=self.additional_features,
                                                                      additional_feature_info=additional_feature_info))
        self.candidate_model = QueryCandidateModel(layer_sizes, MovieModel(unique_movie_titles,
                                                                           embedding_size=embedding_size,
                                                                           additional_features=self.additional_features,
                                                                           additional_feature_info=additional_feature_info))

        self.retrieval_task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=(
                    movies.apply(tf.data.experimental.dense_to_ragged_batch(128)).map(self.candidate_model)),
            ),
        )

    def compute_loss(self, features, training=False):
        query_embeddings = self.query_model({
            **{k: features[k] for k in self.additional_features}
        })
        movie_embeddings = self.candidate_model({
            'movie_title': features['movie_title'],
            **{k: features[k] for k in self.additional_features}
        })
        return self.retrieval_task(query_embeddings, movie_embeddings, compute_metrics=not training)

In [26]:
def train_movielens_model(additional_features):
  trainset = (ratings
              .take(80_000)
              .shuffle(100_000)
              .apply(tf.data.experimental.dense_to_ragged_batch(2048))
              .cache())
  testset = (ratings
              .skip(80_000)
              .take(20_000)
              .apply(tf.data.experimental.dense_to_ragged_batch(2048))
              .cache())
  model = MovieLensModel(layer_sizes, movies, unique_movie_titles, embedding_size,
                         additional_features=additional_features,
                         additional_feature_info=additional_feature_info)

  model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1), run_eagerly=True)
  model_history = model.fit(
      trainset,
      validation_data=testset,
      validation_freq=5,
      epochs=num_epochs,
      verbose=1)

  model.task = tfrs.tasks.Retrieval()
  model.compile()

  return model, model_history.history

In [29]:
# Hyper Params

num_epochs = 25
embedding_size = 32
layer_sizes = tuple([32])
additional_features_set = set([
    # Movie Features
    'movie_title_text',
    'movie_genres',

    # User Features
    'bucketized_user_age',
    'user_gender',
    'user_occupation_label',
])

retrieval_weight = 1
rating_weight = 0

In [ ]:
# Train the model (slow)

model, model_history = train_movielens_model(additional_features_set)

In [46]:
index = tfrs.layers.factorized_top_k.BruteForce()
index.index_from_dataset(movies.apply(tf.data.experimental.dense_to_ragged_batch(100)).map(model.candidate_model))

In [47]:
user_gender_to_feature_value = {
    'Male': True,
    'Female': False,
}

bucketized_user_age_to_feature_value = {
    "Under 18": 1,
    "18-24": 18,
    "25-34": 25,
    "35-44": 35,
    "45-49": 45,
    "50-55": 50,
    "56+": 56
}

user_occupation_label_to_feature_value = {
    "other": 0,
    "artist": 2,
    "college/grad student": 4,
    "farmer": 8,
    "homemaker": 9,
    "K-12 student": 10,
    "programmer": 12,
    "scientist": 15,
    "technician/engineer": 17,
}

In [81]:
# Get recs for custom user input.

user_gender = 'Male' # @param ["Male", "Female"]
bucketized_user_age = '56+' # @param ["Under 18", "18-24", "25-34", "35-44", "45-49", "50-55", "56+"]
user_occupation_label = 'farmer' # @param ["artist", "college/grad student", "farmer", "homemaker", "K-12 student", "programmer", "scientist", "technician/engineer", "other"]

In [82]:
user_details_for_query = {
    'bucketized_user_age': tf.constant([bucketized_user_age_to_feature_value[bucketized_user_age]]),
    'user_gender': tf.constant([user_gender_to_feature_value[user_gender]]),
    'user_occupation_label': tf.constant([user_occupation_label_to_feature_value[user_occupation_label]]),
}

In [83]:
_, titles = index(model.query_model(user_details_for_query))

title_names = np.array([unique_movie_titles[x] for x in titles.numpy().squeeze()])
print(f"Recommendations for user:\n"
      f" {title_names[:5]}")

Recommendations for user:
 [b'Shanghai Noon (2000)' b'What Dreams May Come (1998)'
 b'Mr. Wrong (1996)' b"Farmer's Wife, The (1928)" b'Disclosure (1994)']
